## マルチエージェント: Semantic Kernel サンプル (日本語版)
<!--
TRANSLATION_METADATA:
{
  "source_file": "08-multi-agent/code_samples/08-semantic-kernel.ipynb",
  "language": "ja",
  "translated_at": "2025-08-28T00:00:00Z",
  "translator": "human-reviewed"
}
-->
英語版 `08-semantic-kernel.ipynb` を日本語化。機能は同一です。

目的: Semantic Kernel の AgentGroupChat で選択(Selection)／終了(Termination)関数をプロンプトベースで定義し会話進行・完了判定を行う最小例。

In [1]:
import os
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (KernelFunctionSelectionStrategy, KernelFunctionTerminationStrategy)
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import KernelFunctionFromPrompt

In [2]:
def _create_kernel_with_chat_completion() -> Kernel:
    kernel = Kernel()
    client = AsyncOpenAI(api_key=os.environ['GITHUB_TOKEN'], base_url='https://models.inference.ai.azure.com/')
    kernel.add_service(OpenAIChatCompletion(ai_model_id='gpt-4o-mini', async_client=client))
    return kernel

In [7]:
async def main():
    # === 会話長制御用定数 ================================================
    MIN_AGENT_TURNS = 5  # ユーザ発話を除くエージェント同士のターン数の最低保証

    # === エージェント設定 ==================================================
    REVIEWER_NAME = 'Concierge'
    REVIEWER_INSTRUCTIONS = f"""
    あなたはホテルのコンシェルジュです。旅行者に最もローカルで本物の体験を提供させる視点から提案を評価します。
    承認(approved/approve 等)は以下 3 条件をすべて満たすまで出してはいけません:
      1. 旅行計画に有名ランドマーク + ローカル/非観光体験が 2 種類以上含まれる
      2. 行程が時間帯 or 体験カテゴリで差別化（例: 朝市・小規模ギャラリー・職人ワークショップ 等）
      3. エージェント間ターン数(ユーザを除く) が {MIN_AGENT_TURNS} 以上
    改善指摘では具体的固有名詞を増やしすぎず“方向性”を短く述べ、次の改善ポイントを 1 つに絞る。
    条件未達の段階では approval 表現を絶対に含めない。
    """
    agent_reviewer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    FRONTDESK_NAME = 'FrontDesk'
    FRONTDESK_INSTRUCTIONS = f"""
    あなたは 10 年経験のフロントデスク旅行プランナーです。次を段階的に行います:
      - 1回目: 代表的でやや一般的な観光アイデアを 1 件
      - 2回目以降: コンシェルジュの指摘を 1 点だけ反映し、ローカル性/独自性を徐々に強化
      - 最終形(承認前): 有名スポット + ローカル体験 >=2 (例: 朝市, 路地裏ギャラリー, 小規模工房 見学 等) を含む 具体性とバランス
    各応答は 1 提案 (短い行程またはまとめ) のみで冗長説明を避ける。
    まだ承認されていない段階で自分から承認を促さない。
    """
    agent_writer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=FRONTDESK_NAME,
        instructions=FRONTDESK_INSTRUCTIONS,
    )

    # === 終了判定プロンプト (Termination) ==================================
    termination_function = KernelFunctionFromPrompt(
        function_name='termination',
        prompt=f"""
        会話が完了したか(y/n)を lower-case で 1 語だけ返してください。
        完了 = 以下をすべて満たす:
          - コンシェルジュの直近メッセージに approved / approve / 承認 など肯定承認語が含まれる
          - エージェント(ユーザ除く)の総ターン数が {MIN_AGENT_TURNS} 以上
        上記を満たさない場合は no。

        履歴:
        {{$history}}
        """,
    )

    # === 次発話者選択プロンプト (Selection) ================================
    selection_function = KernelFunctionFromPrompt(
        function_name='selection',
        prompt=f"""
        次に発話すべきエージェント名を 1 語のみで出力してください。
        交互原則: {FRONTDESK_NAME} ↔ {REVIEWER_NAME} を交互。連続発話禁止。
        まだ完了していない場合は最小 {MIN_AGENT_TURNS} ターン到達まで交互継続。
        履歴:
        {{{{$history}}}}
        """,
    )

    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]).strip().lower() == 'yes',
            history_variable_name='history',
            maximum_iterations=12,
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]) if result.value is not None else FRONTDESK_NAME,
            agent_variable_name='agents',
            history_variable_name='history',
        ),
    )

    user_input = 'パリで 1 日の旅行プランを作ってください。最初は有名所からで良いので徐々にローカル体験を増やしたいです。'
    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
    print(f'# User: {user_input}')
    print(f'# 最低ターン保証 (エージェントのみ): {MIN_AGENT_TURNS}')

    conversation = []
    last_content_fingerprint = None

    def _fp(text: str) -> str:
        return str(hash(text))

    def _truncate(s: str, max_len=90):
        return s if len(s) <= max_len else s[: max_len - 3] + '...'

    async def _run_with_retry(retries=3):
        import asyncio
        for attempt in range(1, retries + 1):
            try:
                async for content in chat.invoke():
                    yield content
                break
            except Exception as e:
                if attempt == retries:
                    print(f"[ERROR] 再試行失敗: {type(e).__name__}: {e}")
                    raise
                backoff = 2 ** (attempt - 1)
                print(f"[WARN] 内部エラー ({type(e).__name__}) 発生: {e} -> {backoff}s 後に再試行 ({attempt}/{retries})")
                await asyncio.sleep(backoff)

    turn = 0
    async for content in _run_with_retry():
        body = content.content or ''
        fp = _fp(body)
        if fp == last_content_fingerprint:
            continue
        last_content_fingerprint = fp
        turn += 1
        role = content.name or '*'
        conversation.append({'turn': turn, 'role': role, 'content': body})
        print(f"[{turn:02d}] {role:<10} | {_truncate(body)}")

    print(f"# IS COMPLETE: {chat.is_complete}")

    if conversation:
        print('\n=== FULL TRANSCRIPT ===')
        for item in conversation:
            print(f"\n[Turn {item['turn']:02d}] {item['role']}\n{item['content']}")
        print('\n--- 要約 ---')
        approved = any(('approved' in c['content'].lower()) or ('承認' in c['content']) for c in conversation if c['role'] == REVIEWER_NAME)
        print(f"承認表現検出: {'はい' if approved else 'いいえ'}")
        print(f"総ターン数(エージェント): {len(conversation)}  / 目標 >= {MIN_AGENT_TURNS}")
        if len(conversation) < MIN_AGENT_TURNS:
            print('※ 早期終了: モデルが指示に反して早期承認した可能性があります。温度調整や再実行を検討してください。')

await main()

# User: パリで 1 日の旅行プランを作ってください。最初は有名所からで良いので徐々にローカル体験を増やしたいです。
# 最低ターン保証 (エージェントのみ): 5
[01] FrontDesk  | 朝、エッフェル塔を訪れ、その美しい景色を楽しんだ後、シャンゼリゼ通りを散策しながらルーヴル美術館へ向かいましょう。午前中は、有名なモナリザや名作の数々を鑑賞してください。午後...
[02] Concierge  | その後は、地元のアートシーンを体験するために、マレ地区へ移動して小さなギャラリーやアトリエを訪れましょう。アートの後は、近くのピカソ美術館を訪れて、その作品を楽しむのも良いで...
[03] FrontDesk  | 朝、エッフェル塔を訪れ、その美しい景色を楽しんだ後、シャンゼリゼ通りを散策しながらルーヴル美術館へ向かいましょう。午前中は、有名なモナリザや名作の数々を鑑賞してください。その...
[04] Concierge  | このプランはエッフェル塔やルーヴル美術館といった有名観光地を含み、アートギャラリー訪問や市場体験のようなローカル要素も取り入れられており、非常に良いスタート地点です。しかし、...
[05] FrontDesk  | 朝、エッフェル塔を訪れ、その美しい景色を楽しんだ後、シャンゼリゼ通りを散策しながらルーヴル美術館へ向かいます。午前中には、有名なモナリザや名作を鑑賞してください。ランチには、...
[06] Concierge  | 午前中の有名観光地と午後のローカル体験のバランスが整ってきましたが、依然として旅行プランは条件を満たしていません。

具体的には、午後の活動を特にローカルな体験に絞り込み、さ...
[07] FrontDesk  | 朝、エッフェル塔を訪れた後は、シャンゼリゼ通りを散策し、ルーヴル美術館を見学します。午後はランチに地元のビストロで料理を楽しんだ後、マレ地区に移動し、小さなアートギャラリーや...
[08] Concierge  | 申し訳ありませんが、今のプランは条件を満たしていません。さらに、有名観光地とローカル体験のバランスを取りつつ、時間帯や体験の差別化を図る必要があります。

例えば、以下の修正...
[09] FrontDesk  | 朝はエッフェル塔を訪れ、その後シャンゼリゼ通りを

### 補足: 単一 LLM との最大の違い（要点）
- 役割分担: 提案担当(FrontDesk) と 評価担当(Concierge) の責務切り出しで思考工程を可視化。
- 外部ガバナンス: 発話選択/終了条件を明示プロンプト化し、モデル任せではない制御フローを構築。
- 漸進改善ループ: 評価→単一点フィードバック→改訂提案の反復でプラン品質を段階的に強化。
- 明示的品質ゲート: “承認条件” (多様性/構造/ターン数) を満たすまで承認禁止で早期収束を防止。
- 教育的価値: 中間生成物が残るため改善跡が学習素材となり、単発生成より説明責任が高い。